In [1]:
function int_to_binarray(x::Int,y::Int)
    out = Array{Int8}(undef, y)
    t = x
    for i in 1:y
        out[i] = t % 2
        t ÷= 2
    end
    return out
end

int_to_binarray (generic function with 1 method)

In [2]:
int_to_binarray(32, 8)

8-element Vector{Int8}:
 0
 0
 0
 0
 0
 1
 0
 0

In [3]:
struct Problem
    x1::Vector{Int8}
    x2::Vector{Int8}
    y::Vector{Int8}
end
function Problem(x1::Number, x2::Number, siz::Number)
    x1_in = Int64(x1)
    x2_in = Int64(x2)
    siz_in = Int64(siz)
    y_in = x1_in + x2_in
    return Problem(int_to_binarray(x1_in, siz_in), int_to_binarray(x2_in, siz_in), int_to_binarray(y_in, siz_in))
end

function Problem(x1::Number, x2::Number)
    return Problem(x1, x2, 8)
end

Problem

In [4]:
t = Problem(162, 191)

Problem(Int8[0, 1, 0, 0, 0, 1, 0, 1], Int8[1, 1, 1, 1, 1, 1, 0, 1], Int8[1, 0, 0, 0, 0, 1, 1, 0])

Now we need to start constructing a neural network. The simple neural network will be a vector of nodes with a dense matrix of weights. This will probably turn into a sparse matrix for large numbers of nodes, but with a small number of nodes it's faster to do it as a dense matrix.

In [17]:
using LinearAlgebra
n = 50
NodeVector = rand(Float16, n)
ConnectionMatrix = rand(Float16, (n,n))
ConnectionMatrix .-= 0.5

50×50 Matrix{Float16}:
  0.04492    0.1035     0.3477     0.1641   …  -0.498     -0.127    -0.3389
  0.04883   -0.1904    -0.4727    -0.06152     -0.001953   0.3232    0.2783
 -0.0918     0.1592    -0.1436     0.335       -0.02734   -0.0996    0.457
  0.2295     0.1729     0.4023     0.414       -0.3203     0.01563   0.4922
  0.0791     0.21       0.457     -0.1133      -0.2158    -0.253    -0.2441
 -0.3691    -0.1406    -0.1875     0.497    …   0.3135    -0.3076   -0.4854
  0.2031     0.2695     0.2139     0.3066       0.04395   -0.0762    0.1797
 -0.2607     0.1074    -0.2012     0.1982      -0.415     -0.169     0.4766
 -0.009766  -0.12305    0.02734    0.1543       0.419      0.3018   -0.1826
 -0.2812    -0.001953   0.207      0.02148      0.06445   -0.2783    0.01563
  0.2852    -0.3945    -0.209     -0.03418  …  -0.0635     0.0752   -0.2354
 -0.4785     0.336     -0.4316     0.338        0.4873    -0.08203  -0.2451
 -0.4014     0.378      0.01074    0.2002      -0.4795     0.373 

In [6]:
function increment_network(vec, mat, func, n_static)
    temp = mat * vec
    for i in 1:length(vec)
        if i <= n_static
            temp[i] = vec[i]
        else
            temp[i] = func(temp[i])
        end
    end
    return temp
end
σ = (x)->atan(x)*2/π

#1 (generic function with 1 method)

In [7]:
vec = NodeVector
for i = 1:100000
    vec = increment_network(vec, ConnectionMatrix, σ, 10)
end
vec

50-element Vector{Float16}:
  0.837
  0.5996
  0.5254
  0.709
  0.4863
  0.5205
  0.882
  0.7207
  0.1514
  0.756
 -0.004063
 -0.447
 -0.09
  ⋮
 -0.3225
 -0.571
 -0.2993
 -0.3564
  0.5527
 -0.3003
  0.4148
 -0.459
  0.5137
 -0.406
  0.1427
 -0.4307

In [13]:
vec = increment_network(vec, ConnectionMatrix, σ, 10)
vec

50-element Vector{Float16}:
  0.837
  0.5996
  0.5254
  0.709
  0.4863
  0.5205
  0.882
  0.7207
  0.1514
  0.756
 -0.005676
 -0.4475
 -0.0891
  ⋮
 -0.323
 -0.5703
 -0.2986
 -0.356
  0.553
 -0.3003
  0.4146
 -0.4592
  0.514
 -0.407
  0.142
 -0.4307

In [15]:
function EnergyFunction(v, cm, f, n_static)
    return sum((increment_network(v,cm,f,n_static).-v).^2)
end

EnergyFunction (generic function with 1 method)

In [19]:
vec = NodeVector
EnergyFunction(vec, ConnectionMatrix, σ, 10)

Float16(21.73)

In [33]:
function ConnectionEnergy(cm, ϵ)
    return sum(cm .* tanh.(ϵ.*cm))
end

ConnectionEnergy (generic function with 1 method)

In [20]:
a = 

2×2 Matrix{Int64}:
 2  4
 5  1

In [38]:
ConnectionEnergy(ConnectionMatrix, 1000)

Float16(633.0)